In [0]:
import os
os.chdir('drive/My Drive/Colab Notebooks/Project/AIFrenz_Season2')

In [0]:
pwd

'/content/drive/My Drive/Colab Notebooks/Project/AIFrenz_Season2'

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split , KFold

Using TensorFlow backend.


In [0]:
np.random.seed(777)
tf.set_random_seed(777)

In [0]:
# Load test_images
path = './data/dataset/' 
test_image = np.load(path+'test_images.npy')
test_image.shape

# 평가지표

In [0]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

def mae_over_fscore(y_true, y_pred):
    '''
    y_true: sample_submission.csv 형태의 실제 값
    y_pred: sample_submission.csv 형태의 예측 값
    '''

    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    # 실제값이 0.1 이상인 픽셀의 위치 확인
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    # 실제 값에 결측값이 없는 픽셀의 위치 확인 
    IsNotMissing = y_true >= 0
    
    # mae 계산
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    # f1_score 계산 위해, 실제값에 결측값이 없는 픽셀에 대해 1과 0으로 값 변환
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    # f1_score 계산    
    f_score = f1_score(y_true, y_pred) 
    
    # f1_score가 0일 나올 경우를 대비하여 소량의 값 (1e-07) 추가 
    return mae / (f_score + 1e-07) 

# 시각화

In [0]:
num_layer = len(model.layers)
print('# of Layers:',num_layer)
for layer in model.layers:
    print(layer.name)

# 각 Layer feature map 시각화

In [0]:
model = load_model('U_Net_200520_ver_1_1.h5')

In [0]:
def show_conv_layer(test_data,test_idx,num_layer):
    # 데이터 전처리
    img = test_data[test_idx,:,:,:9]
    img_tensor = np.expand_dims(img,axis=0)
    plt.matshow(img_tensor[test_idx,:,:,5]) # 5: 시각화 할 이미지의 채널
    plt.show()

    # 입력 텐서와 출력 텐서의 리스트로 모델 객체 만들기
    layer_outputs = [layer.output for layer in model.layers[:num_layer]][1:]
    activation_model = Model(inputs=model.input, outputs=layer_outputs)

    # 예측 모드로 모델 실행
    activations = activation_model.predict(img_tensor)

    # 각 layer명 추출
    layer_names = []
    for layer in model.layers[:num_layer]:
        layer_names.append(layer.name)

    images_per_row = 16 # 1 row에 들어갈 이미지 수 

    # 이미지 삽입 그리드 초기화
    for layer_name , layer_activation in zip(layer_names,activations):
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // images_per_row
        display_grid = np.zeros((size*n_cols,images_per_row*size))

        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0,:,:,col*images_per_row + row]
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image,0,255).astype('uint8')
                display_grid[col*size:(col+1)*size,row*size:(row+1)*size] = channel_image

        scale = 1. / size
        plt.figure(figsize=(scale*display_grid.shape[1] , scale*display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid,aspect='auto',cmap='viridis')
    plt.show()

In [0]:
num_layer = len(model.layers)
test_idx = np.random.randint(2416)
show_conv_layer(test_image,test_idx,num_layer)

### 개별 모듈

In [0]:
# 개별 이미지 전처리
img = test_image[10,:,:,:9]
img_tensor = np.expand_dims(img,axis=0)
print(img_tensor.shape)
plt.matshow(img_tensor[0,:,:,5])
plt.show()

In [0]:
# 입력 텐서와 출력 텐서의 리스트로 모델 객체 만들기
layer_outputs = [layer.output for layer in model.layers[:15]][1:]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

# 예측 모드로 모델 실행
activations = activation_model.predict(img_tensor)

In [0]:
# 첫번째 합성곱층의 활성값
first_layer_activation = activations[0]
print(first_layer_activation.shape)
# 20번째 채널 시각화
plt.matshow(first_layer_activation[0,:,:,1],cmap='viridis')
# 16번째 채널 시각화
plt.matshow(first_layer_activation[0,:,:,2],cmap='viridis')

In [0]:
# 중간층의 모든 활성화 채널 시각화
layer_names = []
for layer in model.layers[:39]:
    layer_names.append(layer.name)

images_per_row = 16

for layer_name , layer_activation in zip(layer_names,activations):
    n_features = layer_activation.shape[-1]
    size = layer_activation.shape[1]
    print(size)
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size*n_cols,images_per_row*size))

    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,:,:,col*images_per_row + row]
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image,0,255).astype('uint8')
            display_grid[col*size:(col+1)*size,row*size:(row+1)*size] = channel_image

    scale = 1. / size
    print(scale)
    plt.figure(figsize=(scale*display_grid.shape[1],scale*display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid,aspect='auto',cmap='viridis')
plt.show()

# 필터 시각화

필터가 반응하는 시각적 패턴을 그림

특정 conv layer의 한 필터 값을 최대화하는 손실함수를 정의하여 확률적 경사 상승법 적용.

In [0]:
# 필터 시각화를 위한 손실 함수 정의
layer_name = 'conv2d_21'
filter_index = 0

layer_output = model.get_layer(layer_name).output
loss = K.mean(layer_output[:,:,:,filter_index])

# 입력에 대한 손실의 그래디언트 구하기
gradient = K.gradient(loss,model.input)[0]
gradient /= (K.sqrt(K.mean(K.square(gradient))) + 1e-5) #정규화

# 입력값에 대한 넘파이 출력값 추출
iterate = K.function([model.input],[loss,gradient])
loss_value , gradient_value = iterate([np.zeros((1,40,40,9))])

# 확률적 경사상승법을 사용한 손실 최대화
input_img_data = np.random.random((1,40,40,9))*20 + 128.
step = 1.
for i in ragne(40):
    loss_value , gradient_value = iterate([input_img_data])

    input_img_data += gradient_value * step

In [0]:
# 텐서를 이미지로 형태로 변환하는 함수
def deprocess_image(x):
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    x += 0.5
    x = np.clip(x,0,1)

    x *= 255
    x = np.clip(x,0,255).astype('uint8')
    return x

# 필터 시각화 이미지를 만드는 함수
def generate_pattern(layer_name,filter_index,size=150):
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:,:,:,filter_index])

    gradient = K.gradient(loss,model.input)[0]
    gradient /= (K.sqrt(K.mean(K.square(gradient))) + 1e-5)

    iterate = K.function([model.input],[loss,gradient])

    input_img_data = np.random.random((1,size,size,3)) * 20 + 128. # 잡음 섞인 gray scale로 시작

    step = 1.
    for i in ragne(40):
        loss_value , gradient_value = iterate([input_img_data])
        input_img_data += gradient_value * step

    img = input_img_data[0]
    return deprocess_image(img)

In [0]:
# 층에 있는 각 필터에 반응하는 패턴 생성
layer_name = 'block1_conv1'
size = 64
margin = 5

results = np.zeros((8*szie + 7*margin, 8*size + 7*margin, 3), dtype='uint8')

for i in range(8):
    for j in range(8):
        filter_img = generate_pattern(layer_name, i + (j*8),size=size)
        horizontal_start = i * size + i * margin
        horizontal_end = horizontal_start + size
        vertical_start = j * size + j*margin
        vertical_end = vertical_start + size

        results[horizontal_start:horizontal_end,vertical_start,vertical_end,:] = filter_img

plt.figure(figsize=(20,20))
plt.imshow(results)


# 클래스 활성화 히트맵

In [0]:
from keras.preprocessing import image
# 모델에 입력하기 위한 전처리
img_path = 'data'

img = image.load_img(img_path,target_size=(224,224))

x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

In [0]:
pred = model.predict(x)
print('prediced:',decode_predictions(pred,top=3)[0])
np.argmax(pred[0])

In [0]:
# Grad_CAM 알고리즘 
africa_elephant_output = model.output[:,386]

last_conv_layer = model.get_layer('block5_conv3') # 마지막 합성곱층의 특성 맵
gradient = K.gradients(africa_elephant_output,last_conv_layer)[0] # 마지막 특성 맵 출력에 대한 코끼리클래스의 그래디언트
pooled_gradient = K.mean(gradient,axis=(0,1,2)) # 특성 맵 채널별 그래디언트 평균이 담긴 (512,) 크기의 벡터

iterate = K.function([model.input],[pooled_gradient,last_conv_layer.output[0]])

pooled_gradient_value , conv_layer_output_value = iterate([x])

for i in rage(512):
    conv_layer_output_value[:,:,i] *= pooled_gradient_value[i]

heatmap = np.mean(conv_layer_output_value,axis=1)

heatmap = np.maximum(heatmap,0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)

In [0]:
# 원본 이미지에 히트맵 덧붙이기
import cv2

img = cv2.imread(img_path)

heatmap = cv2.resize(heatmap,(img.shape[1],img.shape[0]))

heatmap = np.uint8(255*heatmap)

heatmap = cv2.applyColorMap(heatmap,cv2.COLORMAP_JET)

superimposed_img = heatmap * 0.4 + img

cv2.imwrite('저장경로',superimposed_img)